In [1]:
from datetime import datetime
print(datetime.now().strftime('%d-%m-%Y %I:%M %p'))

02-08-2020 02:51 AM


Last Updated On

## Data Gathering
### For cases across the world

In [2]:
import numpy as np
import requests
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup
import requests
import warnings
warnings.filterwarnings("ignore")

In [3]:
wd_url = "https://www.worldometers.info/coronavirus/"
req= requests.get(wd_url)
data=req.text
soup=BeautifulSoup(data,'html.parser')

table_body = soup.find('tbody')
table_rows = table_body.find_all('tr')

In [4]:
country = []
totalcases = []
newcases = []
totaldeaths = []
newdeaths =[]
totalrecovered = []

for tr in table_rows:
    td = tr.find_all('td')
    country.append(td[1].text)
    totalcases.append(td[2].text.strip())
    newcases.append(td[3].text.strip())
    totaldeaths.append(td[4].text.strip())
    newdeaths.append(td[5].text.strip())
    totalrecovered.append(td[6].text)

In [5]:
df1 = pd.DataFrame({'Country':country, 'TotalCases':totalcases, 'NewCases':newcases, 
                    'TotalDeaths':totaldeaths, 'NewDeaths':newdeaths, 'TotalRecovered':totalrecovered})
df = df1.loc[8:]
df.head()

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered
8,USA,"4,755,622","+49,733","157,722",+975,"2,354,194"
9,Brazil,"2,707,877","+41,579","93,563",+995,"1,884,051"
10,India,"1,751,919","+54,865","37,403",+852,"1,146,879"
11,Russia,"845,443","+5,462","14,058",+95,"646,524"
12,South Africa,"503,290","+10,107","8,153",+148,"342,461"


In [6]:
df_world = pd.DataFrame(df1.loc[7])
df_world.drop(["Country"],inplace=True)
df_world.head()

,7
TotalCases,"17,967,833"
NewCases,"+215,125"
TotalDeaths,"687,259"
NewDeaths,"+4,866"
TotalRecovered,"11,299,839"


Removing symbols and replacing null values from dataframe

In [7]:
def null_values(data):
    data.fillna(0, inplace=True)
    return data
df = null_values(df)

In [8]:
def preprocess(data):
    for cols in data:
        data[cols] = data[cols].map(lambda x: str(x).replace(',',''))
        data[cols] = data[cols].map(lambda x: str(x).replace('+',''))
        data[cols] = data[cols].map(lambda x: str(x).replace('\n',''))
preprocess(df_world)
preprocess(df)

**For cases in India**

In [9]:
res = requests.get("https://www.mohfw.gov.in")
soup = BeautifulSoup(res.content,'lxml') 
table = soup.find_all('table')[0]
df_Ind = pd.read_html(str(table))[0]
state = df_Ind["Name of State / UT"].tolist()
cases = df_Ind["Total Confirmed cases*"].tolist()
cured = df_Ind["Cured/Discharged/Migrated*"].tolist()
deaths = df_Ind["Deaths**"].tolist()

In [10]:
df_Ind = pd.DataFrame({'States':state, 'Cases':cases, 'Cured':cured, 'Death':deaths})
df_Indtotal = df_Ind.loc[[36]]
df_Ind = df_Ind.loc[:(len(df_Ind)-7)]
df_Ind.head()

,States,Cases,Cured,Death
0,Andaman and Nicobar Islands,50,40,0
1,Andhra Pradesh,10002,4599,119
2,Arunachal Pradesh,148,22,0
3,Assam,5831,3762,9
4,Bihar,8153,6104,56


In [11]:
import pandas as pd 
import plotly.express as px

import cufflinks
from plotly.offline import iplot
cufflinks.go_offline()

## Data Visualisation - World

In [12]:
df_world.head()

,7
TotalCases,9361469
NewCases,15900
TotalDeaths,479910
NewDeaths,961
TotalRecovered,5047420


In [13]:
df_world.iplot(kind='bar',title="World's Total Cases, New Cases, Total Deaths, New Deaths & Total Recovered")

In [14]:
df_newcases = df.copy()
df_newcases["NewCases"] = pd.to_numeric(df_newcases["NewCases"])
df_newcases["NewDeaths"] = pd.to_numeric(df_newcases["NewDeaths"])
df_newcases=df_newcases.sort_values(by = ["NewCases"], ascending = False)
df_newcases = df_newcases[(df_newcases['NewCases']>0) & (df_newcases['NewDeaths']>0)]

fig = px.bar(df_newcases, x='Country', y ='NewCases', color='Country',text='NewCases', height=500, width=1500, 
             title ="Today's New Cases Across Countries")
fig.show()

In [15]:
#df["TotalRecovered"] = pd.to_numeric(df["TotalRecovered"])
df=df.sort_values(by = ["TotalRecovered"], ascending = False).head(50)

df.iplot(
    x='Country',
    y=['TotalRecovered'],
    mode='lines',
    size=8,
    title="Top 50 Countries With Recovered Cases")

In [16]:
df_newcases.iplot(
    x='Country',
    y=['NewDeaths','NewCases'],
    mode='lines',
    size=8,
    title="Today's New Cases & New Deaths")

In [17]:
df_totaldeaths = df.copy()
df_totaldeaths["TotalDeaths"] = pd.to_numeric(df_totaldeaths["TotalDeaths"])
df_totaldeaths = df_totaldeaths[df_totaldeaths['TotalDeaths']>150]
df_totaldeaths=df_totaldeaths.sort_values(["TotalDeaths"], axis=0, ascending=[False])
fig = px.bar(df_totaldeaths, x='Country', y = 'TotalDeaths', color='Country', text='TotalDeaths', height=500, 
             width=1200,title ='Number of Total Deaths Across Countries Greater Than 150')
fig.show()

In [18]:
fig = px.pie(df, values='TotalCases',names='Country', labels={'cured':'TotalCases'}, hover_data=['TotalCases'],
            title='Total Number of Cases Countries Suffered')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

## Data Visualisation - India

In [19]:
import plotly.io as pio
pio.templates.default = "plotly_dark"

df_Indtotal.head()

,States,Cases,Cured,Death
36,Total#,456183,258685,14476


In [20]:
df_Indtotal.mean().iplot(kind='bar',
                        layout=dict(title="India's Total Cases, Cured Cases & Total Deaths"))

In [21]:
df_Indcases = df_Ind.copy()
df_Indcases["Cases"] = pd.to_numeric(df_Indcases["Cases"])
df_Indcases=df_Indcases.sort_values(by = ["Cases"], ascending = False)
fig = px.bar(df_Indcases, x='States', y ='Cases', color='States',text='Cases', height=500, width=950, 
             title ='Number of Total Cases in India')
fig.show()

In [22]:
df_Indcured = df_Ind.copy()
df_Indcured["Cured"] = pd.to_numeric(df_Indcured["Cured"])
df_Indcured=df_Indcured.sort_values(by = ["Cured"], ascending = False)
fig = px.bar(df_Indcured, x='States', y ='Cured', color='States',text='Cured', height=500, width=950, 
             title ='Total Cured Cases in Country')
fig.show()

In [23]:
df_Indcured = df_Ind.copy()
df_Indcured["Cases"] = pd.to_numeric(df_Indcured["Cases"])
df_Indcured=df_Indcured.sort_values(by = ["Cases"], ascending = False)
df_Indcured.iplot(
    x='States',
    y=['Cured','Cases'],
    mode='lines',
    size=8,
    layout=dict(title='Rate of Total Cured & Total Cases In India'))

In [24]:
fig = px.pie(df_Indcured, values='Cases',names='States', labels={'cured':'Cases'}, hover_data=['Cases'],
            title="Total Cases India Suffered as per States")
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()